In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data as data_utils
%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read data from file
with open('data.txt', 'r') as f:
    data = f.readlines()

# Define player data
players = []
for i in range(0, len(data), 6):
    player = {
        'defense_ranks': np.array([int(j) for j in data[i].split(',')], dtype=np.float32),
        'defense_encodings': np.array([int(j) for j in data[i+1].split(',')], dtype=np.float32),
        'fantasy_scores': np.array([float(j) for j in data[i+2].split(',')], dtype=np.float32),
        'skill_score': np.array([float(j) for j in data[i+3].split(',')], dtype=np.float32),
        'weeks_encodings': np.array([int(j) for j in data[i+4].split(',')], dtype=np.float32),
        'season_encodings': np.array([int(j) for j in data[i+5].split(',')], dtype=np.float32)
    }
    players.append(player)

# Process inputs and targets for each player
X_train, Y_train, X_dev, Y_dev, X_test, Y_test = [], [], [], [], [], []
 
for player in players[:30]:
    player_inputs = torch.from_numpy(player['defense_ranks']).view(1, 50, 1).float()
    player_inputs = torch.cat((player_inputs, torch.from_numpy(player['defense_encodings']).view(1, 50, 1).float()), dim=2)
    player_inputs = torch.cat((player_inputs, torch.from_numpy(player['skill_score']).view(1, 50, 1).float()), dim=2)
    player_inputs = torch.cat((player_inputs, torch.from_numpy(player['weeks_encodings']).view(1, 50, 1).float()), dim=2)
    player_inputs = torch.cat((player_inputs, torch.from_numpy(player['season_encodings']).view(1, 50, 1).float()), dim=2)
    X_train.append(player_inputs)
    Y_train.append(torch.from_numpy(player['fantasy_scores']).view(1, 50, 1).float())

for player in players[30:40]:
    player_dev_inputs = torch.from_numpy(player['defense_ranks']).view(1, 50, 1).float()
    player_dev_inputs = torch.cat((player_dev_inputs, torch.from_numpy(player['defense_encodings']).view(1, 50, 1).float()), dim=2)
    player_dev_inputs = torch.cat((player_dev_inputs, torch.from_numpy(player['skill_score']).view(1, 50, 1).float()), dim=2)
    player_dev_inputs = torch.cat((player_dev_inputs, torch.from_numpy(player['weeks_encodings']).view(1, 50, 1).float()), dim=2)
    player_dev_inputs = torch.cat((player_dev_inputs, torch.from_numpy(player['season_encodings']).view(1, 50, 1).float()), dim=2)
    X_dev.append(player_dev_inputs)
    Y_dev.append(torch.from_numpy(player['fantasy_scores']).view(1, 50, 1).float())

for player in players[40:41]:
    player_test_inputs = torch.from_numpy(player['defense_ranks']).view(1, 50, 1).float()
    player_test_inputs = torch.cat((player_test_inputs, torch.from_numpy(player['defense_encodings']).view(1, 50, 1).float()), dim=2)
    player_test_inputs = torch.cat((player_test_inputs, torch.from_numpy(player['skill_score']).view(1, 50, 1).float()), dim=2)
    player_test_inputs = torch.cat((player_test_inputs, torch.from_numpy(player['weeks_encodings']).view(1, 50, 1).float()), dim=2)
    player_test_inputs = torch.cat((player_test_inputs, torch.from_numpy(player['season_encodings']).view(1, 50, 1).float()), dim=2)
    X_test.append(player_test_inputs)  
    Y_test.append(torch.from_numpy(player['fantasy_scores']).view(1, 50, 1).float())

# Combine inputs and targets for all players
X_train = torch.cat(X_train, dim=0).to(device)
Y_train = torch.cat(Y_train, dim=0).to(device)
X_dev = torch.cat(X_dev, dim=0).to(device)
Y_dev = torch.cat(Y_dev, dim=0).to(device) 
X_test = torch.cat(X_test, dim=0).to(device)
Y_test = torch.cat(Y_test, dim=0).to(device)
X_train.shape, Y_train.shape, X_dev.shape, Y_dev.shape, X_test.shape, Y_test.shape

(torch.Size([30, 50, 5]),
 torch.Size([30, 50, 1]),
 torch.Size([10, 50, 5]),
 torch.Size([10, 50, 1]),
 torch.Size([1, 50, 5]),
 torch.Size([1, 50, 1]))

In [6]:
# Define the neural network architecture
class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        out = self.relu(out)
        out = self.fc5(out)
        return out

# Set hyperparameters
initial_learning_rate = 0.0004
batch_size = len(X_train)
num_batches = len(X_train) // batch_size  # Only full batches

# Create the model
model = RegressionModel(5, 1500).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=initial_learning_rate)

train_dataset = data_utils.TensorDataset(X_train, Y_train)
train_loader = data_utils.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Set up a learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=750, gamma=0.5)

total = 0
params = list(model.parameters())
for i in params:
    total += len(i)
total

30


12002

In [81]:
# Training loop for each player
epochs = 1500
 
# Training loop
for epoch in range(epochs):
    model.train()  # Set the model in training mode
    total_loss = 0.0

    for batch_X, batch_Y in train_loader:
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_Y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Adjust learning rate using the scheduler
    scheduler.step()

    # Print the average loss for the epoch
    if epoch + 1 == epochs:
        print(f"Epoch {epoch+1}/{epochs}, Average Loss: {total_loss/num_batches}")

# Evaluation on the dev set
model.eval()  # Set the model in evaluation mode
with torch.no_grad():
    dev_outputs = model(X_dev)
    dev_loss = criterion(dev_outputs, Y_dev)
    print(f"Dev Loss: {dev_loss.item()}")

Epoch 150/150, Average Loss: 42.789737701416016
Dev Loss: 47.92681884765625
